# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [93]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
# Import API key
from api_keys import geoapify_key

In [94]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,georgetown,5.4112,100.3354,78.73,89,20,1.14,MY,1722816520
1,1,codrington,-38.2667,141.9667,48.87,62,92,7.20,AU,1722816521
2,2,bredasdorp,-34.5322,20.0403,44.01,71,26,9.78,ZA,1722816523
3,3,bethel,41.3712,-73.4140,75.60,88,0,5.75,US,1722816524
4,4,kodiak,57.7900,-152.4072,53.11,100,100,10.36,US,1722816526


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [95]:
%%capture --no-display

# Configure the map plot
map=city_data_df.hvplot.points("Lng","Lat",geo=True,tiles="OSM",size="Humidity",color="City").opts(xlabel='Longitude', 
ylabel='Latitude')           

# Display the map
map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [103]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

# Drop any rows with null values
cleaned_city_df = city_data_df.dropna()
city_data_df=cleaned_city_df.loc[(cleaned_city_df["Max Temp"]>60)&(cleaned_city_df["Max Temp"]<70)]
city_data_df=city_data_df.loc[(city_data_df["Humidity"]<70)]
city_data_df=city_data_df.loc[(city_data_df["Wind Speed"]>1)&(city_data_df["Wind Speed"]<5)]
city_data_df=city_data_df.loc[(city_data_df["Cloudiness"]<30)]
# Display sample data
city_data_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
53,53,hammerfest,70.6634,23.6821,64.85,68,0,4.61,NO,1722816585
128,128,phalaborwa,-23.9430,31.1411,68.58,21,20,4.83,ZA,1722816680
167,167,susuman,62.7833,148.1667,61.57,40,12,3.85,RU,1722816728
357,357,kalabo,-14.9700,22.6814,61.99,25,4,4.54,ZM,1722816975
412,412,caimancito,-23.7407,-64.5937,65.21,45,0,1.95,AR,1722817051
416,416,sang-e charak,35.8497,66.4369,68.29,31,0,2.48,AF,1722817056
477,477,vershino-darasunskiy,52.3500,115.5167,62.98,63,2,3.76,RU,1722817133
535,535,kyzylorda,44.8528,65.5092,66.22,48,0,4.47,KZ,1722817203
558,558,khovd,48.0056,91.6419,69.15,41,0,2.26,MN,1722817243


### Step 3: Create a new DataFrame called `hotel_df`.

In [104]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=city_data_df[['City', 'Country', 'Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=None

# Display sample data
hotel_df

/var/folders/y5/nqrrkw794fn9ywj6pd4456hc0000gn/T/ipykernel_31716/1400867900.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=None


,City,Country,Lat,Lng,Humidity,Hotel Name
53,hammerfest,NO,70.6634,23.6821,68,None
128,phalaborwa,ZA,-23.9430,31.1411,21,None
167,susuman,RU,62.7833,148.1667,40,None
357,kalabo,ZM,-14.9700,22.6814,25,None
412,caimancito,AR,-23.7407,-64.5937,45,None
416,sang-e charak,AF,35.8497,66.4369,31,None
477,vershino-darasunskiy,RU,52.3500,115.5167,63,None
535,kyzylorda,KZ,44.8528,65.5092,48,None
558,khovd,MN,48.0056,91.6419,41,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [105]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    hotel=requests.get(base_url, params=params).json()

  
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
       hotel_df.loc[index, "Hotel Name"] = hotel["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
hammerfest - nearest hotel: Thon Hotel Hammerfest
phalaborwa - nearest hotel: Hans Merensky Hotel & Estate
susuman - nearest hotel: No hotel found
kalabo - nearest hotel: Golden Lodge Luxury Acomodation
caimancito - nearest hotel: Termas de Aguas Calientes
sang-e charak - nearest hotel: No hotel found
vershino-darasunskiy - nearest hotel: No hotel found
kyzylorda - nearest hotel: Нур
khovd - nearest hotel: Khovd hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
53,hammerfest,NO,70.6634,23.6821,68,Thon Hotel Hammerfest
128,phalaborwa,ZA,-23.9430,31.1411,21,Hans Merensky Hotel & Estate
167,susuman,RU,62.7833,148.1667,40,No hotel found
357,kalabo,ZM,-14.9700,22.6814,25,Golden Lodge Luxury Acomodation
412,caimancito,AR,-23.7407,-64.5937,45,Termas de Aguas Calientes
416,sang-e charak,AF,35.8497,66.4369,31,No hotel found
477,vershino-darasunskiy,RU,52.3500,115.5167,63,No hotel found
535,kyzylorda,KZ,44.8528,65.5092,48,Нур
558,khovd,MN,48.0056,91.6419,41,Khovd hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [110]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE
map_plot = hotel_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    hover_cols=["City", "Hotel Name", "Country"]
).opts(
    title="Hotel Locations and Information",
    frame_width=800,
    frame_height=500,
    tools=['hover'],  # Enable hover tool
    xlabel='Longitude', 
    ylabel='Latitude'
)

# Display the plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)